### **Objective of the Project:**

• To read the olympix_data.csv file into PySpark as a dataframe.

• To find out the number Sports included in Olympic.

• To plot the total number of medals in each Olympic Sport and sort in descending order based on the total no. of medals.

• To find the total number of medals won by each country in Swimming and Skelton.

• To find the number of medals US won yearly.

• To find out the total number of medals won by each country.

• To check for the oldest athlete in the Olympics and country he\she belongs.

In [0]:
pip install pyspark

Python interpreter will be restarted.
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425366 sha256=28938ea6468066ee9173dc10d09212ab9eb8eca5d1ec70b3a9d3b11fe2ea68b1
  Stored in directory: /root/.cache/pip/wheels/57/bd/14/ce9e21f2649298678d011fb8f71ed38ee70b42b94fef0be142
Successfully built pyspark
Python interpreter will be restarted.


In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName('OlympicsData').getOrCreate()

In [0]:
schema = StructType([ 
    StructField('name', 
                StringType(), True), 
    StructField('age', 
                IntegerType(), True), 
    StructField('country', 
                StringType(), True), 
    StructField('year', 
                IntegerType(), True), 
    StructField('Date_Given', 
                StringType(), True),
    StructField('sports', 
                StringType(), True),
    StructField('gold_medal', 
                IntegerType(), True),
    StructField('silver_medal', 
                IntegerType(), True),
    StructField('brone_medal', 
                IntegerType(), True),
    StructField('total_medal', 
                IntegerType(), True),
]) 

To read the olympix_data.csv file into PySpark as a dataframe.

In [0]:
df = spark.read.format('csv').option('header',True).schema(schema).load('dbfs:/FileStore/tables/olympix_data_organized_with_header.csv')

In [0]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- Date_Given: string (nullable = true)
 |-- sports: string (nullable = true)
 |-- gold_medal: integer (nullable = true)
 |-- silver_medal: integer (nullable = true)
 |-- brone_medal: integer (nullable = true)
 |-- total_medal: integer (nullable = true)



In [0]:
df = df.na.drop()
df.show()

+--------------------+---+-------------+----+----------+--------------------+----------+------------+-----------+-----------+
|                name|age|      country|year|Date_Given|              sports|gold_medal|silver_medal|brone_medal|total_medal|
+--------------------+---+-------------+----+----------+--------------------+----------+------------+-----------+-----------+
|      Michael Phelps| 23|United States|2008| 8/24/2008|            Swimming|         8|           0|          0|          8|
|      Michael Phelps| 19|United States|2004| 8/29/2004|            Swimming|         6|           0|          2|          8|
|      Michael Phelps| 27|United States|2012|08-12-2012|            Swimming|         4|           2|          0|          6|
|    Natalie Coughlin| 25|United States|2008| 8/24/2008|            Swimming|         1|           2|          3|          6|
|       Aleksey Nemov| 24|       Russia|2000|10-01-2000|          Gymnastics|         2|           1|          3|     

To find out the number Sports included in Olympic.

In [0]:
distinct_sports_df = df.select('sports').distinct()

distinct_sports_df.show()


+--------------------+
|              sports|
+--------------------+
|          Gymnastics|
|              Tennis|
|Short-Track Speed...|
|              Rowing|
|            Swimming|
|       Alpine Skiing|
|            Biathlon|
|           Athletics|
|          Equestrian|
|            Shooting|
|              Diving|
|         Ski Jumping|
|     Nordic Combined|
|       Speed Skating|
|        Table Tennis|
|            Canoeing|
|           Bobsleigh|
|             Cycling|
|Cross Country Skiing|
|Synchronized Swim...|
+--------------------+
only showing top 20 rows



To plot the total number of medals in each Olympic Sport and sort in descending order based on the total no. of medals.

In [0]:
from pyspark.sql.functions import sum

df_to_plot = df.groupBy('sports').agg(sum('total_medal'))

df_to_plot.display()

sports,sum(total_medal)
Gymnastics,282
Tennis,78
Boxing,188
Short-Track Speed Skating,138
Ice Hockey,384
Rowing,576
Judo,224
Softball,134
Sailing,210
Swimming,765


Databricks visualization. Run in Databricks to view.

To find the total number of medals won by each country in Swimming and Skeleton.

In [0]:
df.groupBy('country','sports').agg(sum('total_medal')).filter(df.sports == 'Swimming').show()

df.groupBy('country','sports').agg(sum('total_medal')).filter(df.sports == 'Skeleton').show()

+-------------------+--------+----------------+
|            country|  sports|sum(total_medal)|
+-------------------+--------+----------------+
|            Croatia|Swimming|               1|
|          Australia|Swimming|             163|
|              Japan|Swimming|              43|
|            Denmark|Swimming|               1|
|           Slovenia|Swimming|               1|
|        South Korea|Swimming|               4|
|          Lithuania|Swimming|               1|
|Trinidad and Tobago|Swimming|               1|
|            Romania|Swimming|               6|
|           Slovakia|Swimming|               2|
|             France|Swimming|              39|
|             Sweden|Swimming|               9|
|            Hungary|Swimming|               9|
|              Italy|Swimming|              16|
|             Norway|Swimming|               2|
|         Costa Rica|Swimming|               2|
|          Argentina|Swimming|               1|
|             Brazil|Swimming|          

To find the number of medals US won yearly.

In [0]:
df.groupBy('year','country').agg(sum('total_medal')).filter(df.country == 'United States').orderBy('year').show()

+----+-------------+----------------+
|year|      country|sum(total_medal)|
+----+-------------+----------------+
|2000|United States|             243|
|2002|United States|              84|
|2004|United States|             265|
|2006|United States|              52|
|2008|United States|             317|
|2010|United States|              97|
|2012|United States|             254|
+----+-------------+----------------+



To find out the total number of medals won by each country.

In [0]:
df.groupBy('country').agg(sum('total_medal')).show()

+--------------+----------------+
|       country|sum(total_medal)|
+--------------+----------------+
|        Russia|             768|
|      Paraguay|              17|
|Chinese Taipei|              20|
|        Sweden|             181|
|       Eritrea|               1|
|     Singapore|               7|
|      Malaysia|               3|
|        Turkey|              28|
|       Germany|             629|
|   Afghanistan|               2|
|         Sudan|               1|
|        France|             318|
|        Greece|              59|
|     Sri Lanka|               1|
|       Algeria|               8|
|          Togo|               1|
|      Slovakia|              35|
|     Argentina|             139|
|       Belgium|              18|
| Great Britain|             322|
+--------------+----------------+
only showing top 20 rows



To check for the oldest athlete in the Olympics and country he\she belongs.

In [0]:
df.select('name', 'age', 'country').sort('age', ascending = [False]).show()

+--------------------+---+-------------+
|                name|age|      country|
+--------------------+---+-------------+
|          Ian Millar| 61|       Canada|
|           Mark Todd| 56|  New Zealand|
|            Mac Cone| 55|       Canada|
|        Nick Skelton| 54|Great Britain|
|       Peter Charles| 52|Great Britain|
|Lesley Thompson-W...| 52|       Canada|
|       Peter Thomsen| 51|      Germany|
|   Mary Thomson-King| 51|Great Britain|
|       Ilario Di Buò| 51|        Italy|
|    Andrew Nicholson| 50|  New Zealand|
|        Jean Teulère| 50|       France|
|     Debbie McDonald| 49|United States|
|     Rajmond Debevec| 49|     Slovenia|
|        Will Simpson| 49|United States|
|         Russ Howard| 49|       Canada|
|        Robert Dover| 48|United States|
|       Kevin Burnham| 47|United States|
|     Harald Stenvaag| 47|       Norway|
|       Peter Thomsen| 47|      Germany|
|   Mary Thomson-King| 47|Great Britain|
+--------------------+---+-------------+
only showing top

In [0]:
df.select('name', 'age', 'country').sort(df.age.desc()).show(1)

+----------+---+-------+
|      name|age|country|
+----------+---+-------+
|Ian Millar| 61| Canada|
+----------+---+-------+
only showing top 1 row

